In [1]:
from utils import *

pd.set_option("display.notebook_repr_html", False) 

Zrob kolumne od badges w ktorej bedzie string z odznakami(Participation Badges, awarded once) (bez spacji), połącz to po user Id z Posts_questions i Posts_answers (w sumie z dwóch ostatnich możesz zostawić tylko UserId i CreationDate) czyli:

1. Funkcja do Badges, która wyjmie userId i kolumne ze stringiem z odznakami(Participation Badges, awarded once) (bez spacji)
2. Funckja do Posts_questions i Posts_answers - wyjmie UserId i CreationDate.
3. Funckja ktora stworzy słownik - godzina i ilosc postow
4. dalej z tego mozna okreslic:
    + jaka godzina jest popularna/najmniej popularna wsrod danej religii ogolnie - wykres, 
    + jaka godzina najczestsza/najrzadsza wsrod uzytkownikow z dana odznaką -wykres,
    + to samo globalnie albo określić jakieś przedziały w stylu 'rano', 'noc', 'pora obiadowa'

In [2]:
buddyzm = Religion('../buddyzm')
islam = Religion('../islam')
judaism = Religion('../judaism')

W tym punkcie naszym celem było zbadanie zależności między czasem tworzenia posta lub dodania odpowiedzi, religią, a odznakami użytkownika. W tym celu potrzebowaliśmy wyodrębnić ramkę danych stworzoną z UserId i kolumny z wszystkimi odznakami danego użytkownika.
Zajmuje się tym funkcja `participation_badges`.

In [28]:
       def participation_badges(religia):
        """funkcja, która tworzy ramkę danych ParticipationBadges, zawierająca kolumnę UserId i kolumnę UserBadges,
        której wiersze to stringi ParticipationBadges danego użytkownika (przyznawanych tylko raz).
        ParticipationBadges są określone na podstawie
        https://meta.stackexchange.com/questions/67397/what-are-the-badges-i-can-earn-on-each-site-and-what-are-the-exact-criteriafor#188731"""
        
        UserBadges = ['Autobiographer', 'Commentator', 'Pundit', 'Enthusiast', 'Fanatic', 'Mortarboard', 'Epic',
                      'Legendary', 'Precognitive', 'Beta', 'Quorum', 'Convention', 'Talkative', 'Outspoken']
        ParticipationBadges = religia.Badges[['UserId','Name']].copy()

        # tworzę ramkę posiadającą jedynie kolumne UserId z unikatowymi UserId

        # tworze nowa kolumne, w której znajda sie wypisane ParticipationBadges oddzielone'-', danego użytkownika
        # (dla każdego użytkownika - po UserId- jest tworzona lista nazw przyznanych mu Badges, jeśli nazwa należy do UserBadges to jest dopisywana do stringa)
        ParticipationBadges=ParticipationBadges[ParticipationBadges['Name'].isin(UserBadges)]
        ParticipationBadges['Name']+='-'
        ParticipationBadges=ParticipationBadges.groupby('UserId').sum().reset_index()
        ParticipationBadges['Name']=ParticipationBadges['Name'].str.slice(0,-1)
        ParticipationBadges['UserId'] = ParticipationBadges['UserId'].astype(int)
        ParticipationBadges=ParticipationBadges.sort_values('UserId')
    
        
        
        return ParticipationBadges

In [67]:
bPBadges =participation_badges(buddyzm)
jPBadges =participation_badges(judaism)
iPBadges =participation_badges(islam)
# iPBadges

Następnie potrzebowaliśmy funkcji, która zwróci ramkę określającą w której godzinie dany post został stworzony, jakiego jest typu (1- pytanie, 2- odpowiedź) oraz Id i odznak użytkownika, który go stworzył. Zaczęliśmy od zdefiniowania funkcji pomocniczej `clean_time`, która wyodrębniała z daty powstania posta jedynie godzinę, w której został stworzony.

In [8]:
def clean_time(date):
    cleaned = re.search('T[0-9]{2}', date).__getitem__(0)
    date = cleaned[1:]
    date = int(date)
    return date

Potem utworzyliśmy intersującą nas funkcję, czyli `pb_date_qa`.

In [69]:
def pb_date_qa(religia):
    Posts = religia.Posts
    
    # wyodrębniamy kolumny z Id użytkownika, datą i typem postu,
    # zmieniamy nazwę kolumny OwnerUserId na UserId, a jej wartości zmieniamy na integer'y - przydatne do merge,
    PostsQA = Posts[ ['OwnerUserId','CreationDate','PostTypeId'] ].copy()
    PostsQA.rename(columns = {'OwnerUserId':'UserId'},inplace =True)
    PostsQA.dropna(inplace=True)
    PostsQA ['UserId'] = PostsQA['UserId'].astype(int)
    
    # łączymy ramkę PostsQA z ramką zawierającą Id użytkownika i jego odznaki
    PBadges = participation_badges(religia)
    Merged = pd.merge(PBadges, PostsQA ,how='inner', on='UserId')
    
    # wyodrębniamy godzinę utworzenia posta z daty i zmieniamy nazwę tej kolumny na pasującą
    Merged['CreationDate']=Merged['CreationDate'].apply(clean_time)
    Merged.rename(columns={'CreationDate':'CreationHour'}, inplace=True)
    
    return Merged

Korzystając z powyższej funkcji utworzyliśmy ramki danych dla każdej z religii. Na nich możemy już przeprowadzać analizę

In [65]:
buddyzmInspect=pb_date_qa(buddyzm)
islamInspect =pb_date_qa(islam)
judaizmInspect=pb_date_qa(judaism)

Do uzyskania odpowiedzi na nasze pytania przydadzą się następujące funkcje:
+ `popular_hours`
+ `questions_hours`- wersja `popular_hours` dla pytań
+ `answers_hours` - wersja `popular_hours` dla odpowiedzi
+ `sum_badges` wraz z funkcją pomocniczą `search_in_column`
+ `hours_among_badge`

Funkcja `popular_hours` określa ile postów (zarówno pytań jak i odpowiedzi) powstało w danej godzinie.

In [71]:
def popular_hours(religiaInspect):
    summarised = religiaInspect['CreationHour'].value_counts()
   
    return summarised

In [81]:
def questions_hours(religiaInspect):
    return popular_hours(religiaInspect[religiaInspect['PostTypeId']=='1'])

In [86]:
def answers_hours(religiaInspect):
    return popular_hours(religiaInspect[religiaInspect['PostTypeId']=='2'])

`search_in_columns` wyszukuje w danym stringu wyrażenie regularne i zwraca wartość logiczną, w zależńości od tego, czy je znajdzie. Funkcję tą można łatwo wykonać na kolumnie ramki danych, przy użyciu `apply()`

In [62]:
def search_in_column(string,expression):
    return bool(re.search(expression,string))

Poniższa funkcja określa ile postów utworzyli użytkownicy posiadający daną odznakę.

In [63]:
def sum_badges(religiaInspect):
    UserBadges = ['Autobiographer', 'Commentator', 'Pundit', 'Enthusiast', 'Fanatic', 'Mortarboard', 'Epic','Legendary', 'Precognitive', 'Beta', 'Quorum', 'Convention', 'Talkative', 'Outspoken']
    dict={}
    for i in UserBadges:
        df=religiaInspect[religiaInspect['Name'].apply(search_in_column, args=(i,))]
        count=df['Name'].count()
        if count!=0:
            dict[i]=count
    return pd.Series(dict)


Poniższa funkcja określa ile postów w danej godzinie dodały osoby z konkretną odznaką.

In [153]:
def hours_among_badge(religiaInspect,badge):
    return popular_hours(religiaInspect[religiaInspect['Name'].apply(search_in_column, args=(badge,))])

Oprócz tego zdefiniowaliśmy 2 funkcje tworzące interaktywne wykresy kołowe, do prezentacji nasyzch wyników. <br>
Są to mianowicie:
+ `plot_pie` - funkcja ogólna, tworząca wykres na podstawie kolekcji i tytułu, który chcemy nadać wykresowi
+ `plot_pie_grouped` - wersja funkcji `plot_pie` dla zgrupowanych godzin

In [147]:
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go

def plot_pie(series, title):
    series.sort_index(inplace=True)
    lab = series.keys().tolist()
    val = series.tolist()
    trace = go.Pie(labels=lab, 
            values=val, 
            marker=dict(colors=['red']), 
            # Seting values to 
            hoverinfo="label+value+text",
            hovertext ='liczba postów:',
            sort=False

            )
    data = [trace]
    layout = go.Layout(title=title)
    fig = go.Figure(data = data,layout = layout)
    iplot(fig)


In [148]:
def plot_pie_grouped(series, title):
    # Tworzymy 6 grup godzin utworzenia postów: <2-6), <6-10), <10-14), <14-18), <18-22),<22-2)
    lab1=[22,23,0,1]
    lab2=[2,3,4,5]
    lab3= [6,7,8,9]
    lab4=[10,11,12,13]
    lab5= [14,15,16,17]
    lab6= [18,19,20,21]

    trace = go.Pie(labels =['2-5','6-9','10-13','14-17','18-21','22-1'],
            values=[sum(series.filter(lab2)),sum(series.filter(lab3)),sum(series.filter(lab4)),sum(series.filter(lab5)),sum(series.filter(lab6)),sum(series.filter(lab1))], 
            marker=dict(colors=['red']), 
            # Seting values to
            hovertext ='liczba postów:',
            hoverinfo="label+value+text",
            sort=False
            )
   
    data = [trace]
    layout = go.Layout(title=title)
    fig = go.Figure(data = data,layout = layout)
    iplot(fig)


## Ile postów powstaje w danej godzinie na każdym forum?

In [149]:
plot_pie(popular_hours(buddyzmInspect),"Popularność godzin dodawania postów na forum buddyzmu")
plot_pie(popular_hours(islamInspect),"Popularność godzin dodawania postów na forum islamu")
plot_pie(popular_hours(judaizmInspect),"Popularność godzin dodawania postów na forum judaizmu")

Trudno wnioskować z wykresu z tak dużą ilością etykiet, ponieważ żadna wyraźnie się nie wyróżnia. Zobaczmy czy po podzieleniu godzin na przedziały czasowe uzyskamy ciekawe wyniki.

### Po dodaniu przedziałów czasowych

In [150]:
plot_pie_grouped(popular_hours(buddyzmInspect),"Popularność godzin dodawania postów na forum buddyzmu, w przedziałach czasowych")
plot_pie_grouped(popular_hours(islamInspect),"Popularność godzin dodawania postów na forum islamu, w przedziałach czasowych")
plot_pie_grouped(popular_hours(judaizmInspect),"Popularność godzin dodawania postów na forum judaizmu, w przedziałach czasowych")

## Użytkownicy z jakimi odznakami dodali najwięcej postów?

In [151]:
plot_pie(sum_badges(buddyzmInspect),"Rozkład postów pomiędzy poszczególnymi odznakami na forum buddyzmu")
plot_pie(sum_badges(islamInspect),"Rozkład postów pomiędzy poszczególnymi odznakami na forum islamu")
plot_pie(sum_badges(judaizmInspect),"Rozkład postów pomiędzy poszczególnymi odznakami na forum judaizmu")

### A jak to się przedstawia w podziale na pytania i odpowiedzi?

In [152]:
plot_pie(sum_badges(buddyzmInspect[buddyzmInspect.PostTypeId=='1']),"Rozkład dodanych pytań pomiędzy poszczególnymi odznakami na forum buddyzmu")
plot_pie(sum_badges(buddyzmInspect[buddyzmInspect.PostTypeId=='2']),"Rozkład dodanych odpowiedzi pomiędzy poszczególnymi odznakami na forum buddyzmu")

plot_pie(sum_badges(islamInspect[islamInspect.PostTypeId=='1']),"Rozkład dodanych pytań pomiędzy poszczególnymi odznakami na forum islamu")
plot_pie(sum_badges(islamInspect[islamInspect.PostTypeId=='2']),"Rozkład dodanych odpowiedzi pomiędzy poszczególnymi odznakami na forum islamu")

plot_pie(sum_badges(judaizmInspect[judaizmInspect.PostTypeId=='1']),"Rozkład dodanych pytań pomiędzy poszczególnymi odznakami na forum judaizmu")
plot_pie(sum_badges(judaizmInspect[judaizmInspect.PostTypeId=='2']),"Rozkład dodanych odpowiedzi pomiędzy poszczególnymi odznakami na forum judaizmu")

In [159]:
plot_pie_grouped(hours_among_badge(buddyzmInspect,'Fanatic'),"O której godzinie postują fanatycy fora o buddyźmie?")